## Prerequisites

 * Load corpus data

In [1]:
import sys
sys.path.append("..")

from collections import defaultdict
from chaininglib.search.metadata import get_available_metadata
from chaininglib.ui.dfui import display_df, save_dataframe
from chaininglib.search.CorpusQuery import *


corpus_name="zeebrieven"
query=r'[pos="NOU"]'
fields = get_available_metadata(corpus_name)
chosen_fields = ["witnessYear_from", "witnessYear_to", "afz_klasse","afz_geslacht"]
c = create_corpus(corpus_name).pattern(query).extra_fields_doc(chosen_fields).search()
df_corpus = c.kwic()

save_dataframe(df_corpus, "zeebrieven_case_study.csv")






ValueError: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured when searching corpus zeebrieven: An error occured wh

 * Compute frequencies

In [2]:
from chaininglib.utils.dfops import df_filter
from chaininglib.process.corpus import get_frequency_list
from chaininglib.ui.dfui import load_dataframe
import itertools

df_corpus = load_dataframe("zeebrieven_case_study.csv")

df = {}
df["17th"]= df_corpus[df_corpus["witnessYear_from"] < 1700] 
df["18th"] = df_corpus[df_corpus["witnessYear_from"] > 1700] 

eras = ["17th", "18th"]
soc_classes = ["low","high"]
genders = ["male","female"]

# Then, filter Zeebrieven per sociolinguistic variable (social class/occupation)
# Take only extremes: high and low. Remove middle-low and middle-high
freq_list_aggr = defaultdict(lambda: dict())
freq_list = defaultdict(lambda: defaultdict(lambda: dict()))
for era, df_era in df.items():
    for soc_class in soc_classes:
        cond_era_class = df_filter(df_era["afz_klasse"], pattern=soc_class)
        df_era_class = df_era[ cond_era_class ]
        freq_list_aggr[era][soc_class] = get_frequency_list(df_era_class)
        for gender in genders:
            cond_era_gender = df_filter(df_era["afz_geslacht"], pattern=gender)
            df_era_class_gender = df_era[cond_era_class & cond_era_gender ]
            freq_list[era][soc_class][gender] = get_frequency_list(df_era_class_gender)

from chaininglib.utils.dfops import get_relfreq_diff
SHOW=20


zeebrieven_case_study.csv loaded successfully
                                                                    ...Building frequency list...

## Comparisons

### Class difference, era fixed

In [3]:
df_relfreq_diff = {}

for era in eras:
    soc_class_pairs = itertools.combinations(soc_classes,2)
    for soc_class1,soc_class2 in soc_class_pairs:
        df_relfreq_diff = get_relfreq_diff(freq_list_aggr[era][soc_class1], freq_list_aggr[era][soc_class2], index="lemmas", label1=soc_class1, label2=soc_class2 )
        label_df1 = f"Comparison {soc_class1} and {soc_class2} class in {era} century. Words specific for {soc_class1}."
        label_df2 = f"Comparison {soc_class1} and {soc_class2} class in {era} century. Words specific for {soc_class2}."
        d1 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=False).head(SHOW)
        d2 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=True).head(SHOW)
        display_df(d1, labels=label_df1)
        display_df(d2, labels=label_df2)
        save_dataframe(d1, f"{label_df1}.csv", as_latex=True)
        save_dataframe(d2, f"{label_df2}.csv", as_latex=True)


,low,high,relfreq_diff
lemmata,,,
goedenacht,0.026081,0.009204,1.016723
hart,0.026874,0.011850,1.014848
man,0.033023,0.019731,1.013035
brief,0.034113,0.022016,1.011836
Heer,0.021519,0.010647,1.010757
gezondheid,0.025585,0.016181,1.009254
zuster,0.016263,0.008241,1.007957
huisvrouw,0.023701,0.015941,1.007638
kind,0.020329,0.012993,1.007242


,low,high,relfreq_diff
lemmata,,,
kapitein,0.004760,0.012271,0.992580
monsieur,0.000298,0.007700,0.992654
suiker,0.002479,0.009685,0.992863
sinjeur,0.001388,0.006136,0.995282
mijnheer,0.004264,0.008963,0.995343
schipper,0.004958,0.009023,0.995972
heer,0.001983,0.005715,0.996290
pond,0.000496,0.003549,0.996958
december,0.001785,0.004752,0.997047


Comparison low and high class in 17th century. Words specific for low..csv saved
Comparison low and high class in 17th century. Words specific for high..csv saved


,low,high,relfreq_diff
lemmata,,,
gezondheid,0.026068,0.006578,1.019363
vrouw,0.028536,0.010336,1.018014
brief,0.030696,0.013921,1.016545
groetenis,0.019898,0.004559,1.015270
moeder,0.020824,0.007343,1.013382
vriend,0.020824,0.011450,1.009268
zuster,0.021749,0.012946,1.008690
broer,0.010952,0.003515,1.007411
hart,0.014962,0.007865,1.007042


,low,high,relfreq_diff
lemmata,,,
heer,0.010335,0.021090,0.989467
mijnheer,0.009563,0.017679,0.992025
kapitein,0.012803,0.017157,0.995719
mejuffrouw,0.001542,0.005673,0.995893
tante,0.000000,0.003376,0.996636
liefde,0.002931,0.005777,0.997170
missive,0.000308,0.003063,0.997254
goed,0.005090,0.007726,0.997384
reden,0.000617,0.003132,0.997493


Comparison low and high class in 18th century. Words specific for low..csv saved
Comparison low and high class in 18th century. Words specific for high..csv saved


### Class difference, era and gender fixed

In [4]:
df_relfreq_diff = {}
for era in eras:
    for gender in genders:
        soc_class_pairs = itertools.combinations(soc_classes,2)
        for soc_class1,soc_class2 in soc_class_pairs:
            df_relfreq_diff = get_relfreq_diff(freq_list[era][soc_class1][gender], freq_list[era][soc_class2][gender], index="lemmas", label1=soc_class1, label2=soc_class2 )
            label_df1 = f"Comparison {soc_class1} and {soc_class2} class in {era} century with gender {gender}. Words specific for {soc_class1}."
            d1 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=False).head(SHOW)
            label_df2 = f"Comparison {soc_class1} and {soc_class2} class in {era} century with gender {gender}. Words specific for {soc_class2}."
            d2 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=True).head(SHOW)
            display_df(d1, labels=label_df1)
            display_df(d2, labels=label_df2)
            save_dataframe(d1, f"{label_df1}.csv", as_latex=True)
            save_dataframe(d2, f"{label_df2}.csv", as_latex=True)


,low,high,relfreq_diff
lemmata,,,
goedenacht,0.026081,0.009211,1.016716
hart,0.026874,0.011860,1.014839
man,0.033023,0.019746,1.013020
brief,0.034113,0.022034,1.011819
Heer,0.021519,0.010656,1.010749
gezondheid,0.025585,0.016194,1.009241
zuster,0.016263,0.008248,1.007950
huisvrouw,0.023701,0.015953,1.007626
kind,0.020329,0.013003,1.007232


,low,high,relfreq_diff
lemmata,,,
kapitein,0.004760,0.012281,0.992570
monsieur,0.000298,0.007706,0.992648
suiker,0.002479,0.009692,0.992856
sinjeur,0.001388,0.006141,0.995277
mijnheer,0.004264,0.008970,0.995336
schipper,0.004958,0.009030,0.995965
heer,0.001983,0.005719,0.996285
pond,0.000496,0.003552,0.996955
december,0.001785,0.004756,0.997043


Comparison low and high class in 17th century with gender male. Words specific for low..csv saved
Comparison low and high class in 17th century with gender male. Words specific for high..csv saved


,low,high,relfreq_diff
lemmata,,,
hart,0.030536,0.016959,1.013351
goedenacht,0.027098,0.015984,1.010938
Heer,0.023246,0.015205,1.007921
gezondheid,0.027098,0.021832,1.005153
allerliefste,0.006465,0.001365,1.005093
huisvrouw,0.024897,0.020663,1.004148
kenner,0.006878,0.002924,1.003942
brief,0.038102,0.034113,1.003857
mens,0.006190,0.003119,1.003061


,low,high,relfreq_diff
lemmata,,,
kapitein,0.004814,0.012281,0.992624
schipper,0.004402,0.010331,0.994131
man,0.040853,0.046004,0.995075
dochter,0.005365,0.009747,0.995660
vader,0.011417,0.015400,0.996078
vrouw,0.007290,0.010526,0.996798
heer,0.002063,0.004483,0.997591
mijnheer,0.004264,0.006628,0.997652
moeder,0.016369,0.018713,0.997698


...

,low,high,relfreq_diff
lemmata,,,
heer,0.010335,0.021090,0.989467
mijnheer,0.009563,0.017679,0.992025
kapitein,0.012803,0.017157,0.995719
mejuffrouw,0.001542,0.005673,0.995893
tante,0.000000,0.003376,0.996636
liefde,0.002931,0.005777,0.997170
missive,0.000308,0.003063,0.997254
goed,0.005090,0.007726,0.997384
reden,0.000617,0.003132,0.997493


Comparison low and high class in 18th century with gender male. Words specific for low..csv saved
Comparison low and high class in 18th century with gender male. Words specific for high..csv saved


,low,high,relfreq_diff
lemmata,,,
jaar,0.029712,0.016894,1.012606
broer,0.016713,0.005369,1.011283
zuster,0.037140,0.027763,1.009124
groetenis,0.014856,0.005762,1.009042
welstand,0.012999,0.004715,1.008246
brief,0.026927,0.018727,1.008049
moeder,0.017642,0.010215,1.007352
moei,0.008357,0.001179,1.007169
dood,0.010214,0.003798,1.006391


,low,high,relfreq_diff
lemmata,,,
edele,0.000000,0.013620,0.986563
mejuffrouw,0.000000,0.005107,0.994919
mijnheer,0.012999,0.017548,0.995529
mevrouw,0.001857,0.005762,0.996117
dag,0.003714,0.007596,0.996148
compliment,0.000000,0.003012,0.996997
goed,0.000929,0.003929,0.997011
tante,0.000000,0.002881,0.997127
dienares,0.000000,0.002881,0.997127


Comparison low and high class in 18th century with gender female. Words specific for low..csv saved
Comparison low and high class in 18th century with gender female. Words specific for high..csv saved


### Evolution, social class fixed

In [5]:
df_relfreq_diff = {}
for soc_class in soc_classes:
    era_pairs = itertools.combinations(eras,2)
    for era1, era2 in era_pairs:
        df_relfreq_diff = get_relfreq_diff(freq_list_aggr[era1][soc_class], freq_list_aggr[era2][soc_class], index="lemmas", label1=era1, label2=era2 )
        label_df1 = f"Comparison {era1} and {era2} century for class {soc_class}. Words specific for {era1} century."
        d1 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=False).head(SHOW)
        label_df2 = f"Comparison {era1} and {era2} century for class {soc_class}. Words specific for {era2} century."
        d2 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=True).head(SHOW)
        
        display_df(d1, labels=label_df1)
        display_df(d2, labels=label_df2)
        save_dataframe(d1, f"{label_df1}.csv", as_latex=True)
        save_dataframe(d2, f"{label_df2}.csv", as_latex=True)


,17th,18th,relfreq_diff
lemmata,,,
goedenacht,0.026081,0.000000,1.026081
man,0.033023,0.013728,1.019033
huisvrouw,0.023701,0.008175,1.015400
hart,0.026874,0.014962,1.011736
Heer,0.021519,0.010643,1.010761
genade,0.012594,0.001851,1.010723
pas,0.007140,0.000463,1.006674
land,0.009123,0.002468,1.006639
kenner,0.006049,0.000000,1.006049


,17th,18th,relfreq_diff
lemmata,,,
vrouw,0.008132,0.028536,0.980162
zegen,0.002479,0.011723,0.990863
welstand,0.000298,0.009255,0.991125
jaar,0.006743,0.015579,0.991300
broer,0.002380,0.010952,0.991521
heer,0.001983,0.010335,0.991734
januari,0.001488,0.009718,0.991849
kapitein,0.004760,0.012803,0.992059
staat,0.001884,0.009563,0.992393


Comparison 17th and 18th century for class low. Words specific for 17th century..csv saved
Comparison 17th and 18th century for class low. Words specific for 18th century..csv saved


,17th,18th,relfreq_diff
lemmata,,,
huisvrouw,0.015941,0.002436,1.013472
vriend,0.023460,0.011450,1.011874
man,0.019731,0.009675,1.009959
gezondheid,0.016181,0.006578,1.009541
goedenacht,0.009204,0.000139,1.009063
schipper,0.009023,0.000418,1.008602
brief,0.022016,0.013921,1.007984
suiker,0.009685,0.002088,1.007581
monsieur,0.007700,0.001253,1.006439


,17th,18th,relfreq_diff
lemmata,,,
heer,0.005715,0.021090,0.984942
mijnheer,0.008963,0.017679,0.991435
edele,0.000000,0.008387,0.991682
jaar,0.005053,0.012041,0.993095
mejuffrouw,0.000722,0.005673,0.995077
kapitein,0.012271,0.017157,0.995196
zuster,0.008241,0.012946,0.995355
achting,0.000000,0.004524,0.995496
familie,0.001504,0.005916,0.995613


Comparison 17th and 18th century for class high. Words specific for 17th century..csv saved
Comparison 17th and 18th century for class high. Words specific for 18th century..csv saved


### Evolution, social class and gender fixed

In [6]:
df_relfreq_diff = {}
for gender in genders:
    for soc_class in soc_classes:
        era_pairs = itertools.combinations(eras,2)
        for era1, era2 in era_pairs:
            df_relfreq_diff = get_relfreq_diff(freq_list[era1][soc_class][gender], freq_list[era2][soc_class][gender], index="lemmas", label1=era1, label2=era2 )
            label_df1 = f"Comparison {era1} and {era2} century for class {soc_class} and gender {gender}. Words specific for {era1}."
            d1 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=False).head(SHOW)
            label_df2 = f"Comparison {era1} and {era2} century for class {soc_class} and gender {gender}. Words specific for {era2}."
            d2 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=True).head(SHOW)
            
            display_df(d1, labels=label_df1)
            display_df(d2, labels=label_df2)
            save_dataframe(d1, f"{label_df1}.csv", as_latex=True)
            save_dataframe(d2, f"{label_df2}.csv", as_latex=True)

            

,17th,18th,relfreq_diff
lemmata,,,
goedenacht,0.026081,0.000000,1.026081
man,0.033023,0.013728,1.019033
huisvrouw,0.023701,0.008175,1.015400
hart,0.026874,0.014962,1.011736
Heer,0.021519,0.010643,1.010761
genade,0.012594,0.001851,1.010723
pas,0.007140,0.000463,1.006674
land,0.009123,0.002468,1.006639
kenner,0.006049,0.000000,1.006049


,17th,18th,relfreq_diff
lemmata,,,
vrouw,0.008132,0.028536,0.980162
zegen,0.002479,0.011723,0.990863
welstand,0.000298,0.009255,0.991125
jaar,0.006743,0.015579,0.991300
broer,0.002380,0.010952,0.991521
heer,0.001983,0.010335,0.991734
januari,0.001488,0.009718,0.991849
kapitein,0.004760,0.012803,0.992059
staat,0.001884,0.009563,0.992393


Comparison 17th and 18th century for class low and gender male. Words specific for 17th..csv saved
Comparison 17th and 18th century for class low and gender male. Words specific for 18th..csv saved


,17th,18th,relfreq_diff
lemmata,,,
huisvrouw,0.015953,0.002436,1.013484
vriend,0.023478,0.011450,1.011892
man,0.019746,0.009675,1.009974
gezondheid,0.016194,0.006578,1.009554
goedenacht,0.009211,0.000139,1.009070
schipper,0.009030,0.000418,1.008609
brief,0.022034,0.013921,1.008001
suiker,0.009692,0.002088,1.007588
monsieur,0.007706,0.001253,1.006445


,17th,18th,relfreq_diff
lemmata,,,
heer,0.005719,0.021090,0.984947
mijnheer,0.008970,0.017679,0.991442
edele,0.000000,0.008387,0.991682
jaar,0.005057,0.012041,0.993099
mejuffrouw,0.000722,0.005673,0.995078
kapitein,0.012281,0.017157,0.995206
zuster,0.008248,0.012946,0.995361
achting,0.000000,0.004524,0.995496
familie,0.001505,0.005916,0.995615


...

,17th,18th,relfreq_diff
lemmata,,,
jaar,0.006327,0.029712,0.977290
zuster,0.016644,0.037140,0.980238
heer,0.002063,0.019499,0.982898
broer,0.002476,0.016713,0.985997
welstand,0.000413,0.012999,0.987575
mijnheer,0.004264,0.012999,0.991377
broeder,0.013755,0.022284,0.991657
neef,0.001651,0.009285,0.992436
dood,0.002751,0.010214,0.992613


Comparison 17th and 18th century for class low and gender female. Words specific for 17th..csv saved
Comparison 17th and 18th century for class low and gender female. Words specific for 18th..csv saved


,17th,18th,relfreq_diff
lemmata,,,
man,0.046004,0.021870,1.023617
huisvrouw,0.020663,0.000917,1.019728
goedenacht,0.015984,0.000000,1.015984
brief,0.034113,0.018727,1.015103
schip,0.023002,0.008119,1.014763
gezondheid,0.021832,0.009691,1.012025
genade,0.012671,0.002357,1.010289
schipper,0.010331,0.000393,1.009935
moeder,0.018713,0.010215,1.008413


,17th,18th,relfreq_diff
lemmata,,,
edele,0.000000,0.013620,0.986563
heer,0.004483,0.017941,0.986779
zuster,0.014620,0.027763,0.987212
jaar,0.003899,0.016894,0.987221
mijnheer,0.006628,0.017548,0.989268
mevrouw,0.000000,0.005762,0.994271
broeder,0.011111,0.016894,0.994314
familie,0.000390,0.005762,0.994658
liefde,0.002144,0.007465,0.994719


Comparison 17th and 18th century for class high and gender female. Words specific for 17th..csv saved
Comparison 17th and 18th century for class high and gender female. Words specific for 18th..csv saved


### Gender analysis, era and social class fixed

In [7]:
df_relfreq_diff = {}
for era in eras:
    for soc_class in soc_classes:
        gender_pairs = itertools.combinations(genders,2)
        for gender1, gender2 in gender_pairs:
            df_relfreq_diff = get_relfreq_diff(freq_list[era][soc_class][gender1], freq_list[era][soc_class][gender2], index="lemmas", label1=gender1, label2=gender2 )
            label_df1 = f"Comparison {gender1} and {gender2} for {era} century and class {soc_class}. Words specific for {gender1}."
            d1 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=False).head(SHOW)
            label_df2 = f"Comparison {gender1} and {gender2} for {era} century and class {soc_class}. Words specific for {gender2}."
            d2 = df_relfreq_diff.sort_values(by="relfreq_diff", ascending=True).head(SHOW)
            
            display_df(d1, labels=label_df1)
            display_df(d2, labels=label_df2)
            save_dataframe(d1, f"{label_df1}.csv", as_latex=True)
            save_dataframe(d2, f"{label_df2}.csv", as_latex=True)


,male,female,relfreq_diff
lemmata,,,
suiker,0.002479,0.000138,1.002341
schip,0.023602,0.022008,1.001559
oom,0.002975,0.001788,1.001185
vriend,0.022313,0.021183,1.001106
december,0.001785,0.000688,1.001096
maand,0.002876,0.001788,1.001086
huis,0.004363,0.003439,1.000921
port,0.001884,0.000963,1.000920
vaderland,0.001884,0.000963,1.000920


,male,female,relfreq_diff
lemmata,,,
man,0.033023,0.040853,0.992477
brief,0.034113,0.038102,0.996158
hart,0.026874,0.030536,0.996446
Heer,0.021519,0.023246,0.998312
zoon,0.016164,0.017882,0.998313
allerliefste,0.004859,0.006465,0.998405
mens,0.004661,0.006190,0.998480
gezondheid,0.025585,0.027098,0.998527
huisvrouw,0.023701,0.024897,0.998833


Comparison male and female for 17th century and class low. Words specific for male..csv saved
Comparison male and female for 17th century and class low. Words specific for female..csv saved


,male,female,relfreq_diff
lemmata,,,
suiker,0.009692,0.000780,1.008906
monsieur,0.007706,0.001365,1.006333
vriend,0.023478,0.018713,1.004677
sinjeur,0.006141,0.001949,1.004183
december,0.004756,0.000585,1.004169
goed,0.010595,0.006823,1.003747
vracht,0.003191,0.000195,1.002995
mr.,0.002589,0.000000,1.002589
cargasoen,0.002709,0.000195,1.002514


,male,female,relfreq_diff
lemmata,,,
man,0.019746,0.046004,0.974897
brief,0.022034,0.034113,0.988319
kind,0.013003,0.022222,0.990982
zoon,0.010836,0.017934,0.993028
goedenacht,0.009211,0.015984,0.993333
zuster,0.008248,0.014620,0.993719
gezondheid,0.016194,0.021832,0.994482
dochter,0.004395,0.009747,0.994700
genade,0.007585,0.012671,0.994978


...

,male,female,relfreq_diff
lemmata,,,
zuster,0.021749,0.037140,0.985160
jaar,0.015579,0.029712,0.986275
man,0.013728,0.026927,0.987148
kind,0.014808,0.025998,0.989093
heer,0.010335,0.019499,0.991011
broeder,0.014037,0.022284,0.991932
broer,0.010952,0.016713,0.994333
Heer,0.010643,0.015785,0.994939
zoon,0.012494,0.017642,0.994942


Comparison male and female for 18th century and class low. Words specific for male..csv saved
Comparison male and female for 18th century and class low. Words specific for female..csv saved


,male,female,relfreq_diff
lemmata,,,
schip,0.017088,0.008119,1.008896
kapitein,0.017157,0.008905,1.008179
goed,0.007726,0.003929,1.003782
heer,0.021090,0.017941,1.003093
eiland,0.003063,0.000000,1.003063
vat,0.002541,0.000131,1.002409
rede,0.002715,0.000393,1.002321
dienaar,0.002645,0.000393,1.002251
zee,0.004315,0.002095,1.002215


,male,female,relfreq_diff
lemmata,,,
zuster,0.012946,0.027763,0.985583
man,0.009675,0.021870,0.988066
kind,0.011763,0.019513,0.992399
Heer,0.007134,0.014667,0.992576
broeder,0.010789,0.016894,0.993996
zoon,0.006299,0.011655,0.994706
hart,0.007865,0.013227,0.994708
edele,0.008387,0.013620,0.994838
jaar,0.012041,0.016894,0.995228


Comparison male and female for 18th century and class high. Words specific for male..csv saved
Comparison male and female for 18th century and class high. Words specific for female..csv saved
